<a href="https://colab.research.google.com/github/tribecca37/NYC_Jobs/blob/main/nycjob_extracttoStaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage
!pip install requests

In [2]:
# import Librairies
import pandas as pd
import numpy as  np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO

In [3]:
# Function
import os
import boto3
from azure.storage.blob import BlobServiceClient
from google.cloud import storage
import pandas as pd
from io import BytesIO, StringIO

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

# Google Cloud Functions
def google_upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

def google_download_blob(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

# AWS Functions
def aws_upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(file_name, bucket, object_name)
    print(f"Uploaded {file_name} to S3 bucket {bucket}.")

def aws_download_file(bucket, object_name, file_name):
    s3_client = boto3.client('s3')
    s3_client.download_file(bucket, object_name, file_name)
    print(f"Downloaded {object_name} from S3 bucket {bucket}.")

s1 --> Gather --> Deccompress --> Convert to Tabular --> Clean  --> Reformat ---> Consolidation --> Transformation --> Load

In [13]:
import requests
response = requests.get("https://data.cityofnewyork.us/api/odata/v4/kpav-sd4t")
print(response.status_code)

200


In [40]:
import requests
import pandas as pd

# API endpoint URL
url = "https://data.cityofnewyork.us/api/odata/v4/kpav-sd4t"

# Sending a GET request to the API
response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()  # Convert JSON response to a dictionary
    if 'value' in data:
        df_raw = pd.DataFrame(data['value'])
        print(df_raw.head())
        print(df_raw.columns)
        #access only specific columns, you can do it here directly:
        selected_df = df_raw[['job_id', 'agency', 'posting_type', 'number_of_positions', 'business_title']]
        print(df_raw.shape)  # Print the shape of the DataFrame
    else:
        print("No data found in response.")
else:
    print("Failed to fetch data:", response.status_code)
    print(response.text)


                 __id  job_id                          agency posting_type  \
0  row-srwb~tx2x~n8ev  534519  DEPT OF ENVIRONMENT PROTECTION     Internal   
1  row-gmmp~st9n.mrhz  631363  DEPT OF ENVIRONMENT PROTECTION     Internal   
2  row-7ijt_sqje-vi6i  628158         BRONX DISTRICT ATTORNEY     External   
3  row-zgeu~yra9-82sy  629002      OFFICE OF CRIMINAL JUSTICE     Internal   
4  row-atvs.3bjf~45tx  613253   DEPT OF DESIGN & CONSTRUCTION     Internal   

  number_of_positions                                     business_title  \
0                   2                                  Region Supervisor   
1                   1  2024-BWS-010-Early Warning Remote Modeling Int...   
2                   2                               Intelligence Analyst   
3                   1               Senior Coordinator, Capital Projects   
4                   1                                           Director   

              civil_service_title title_classification title_code_no level

In [41]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   __id                           1000 non-null   object
 1   job_id                         1000 non-null   object
 2   agency                         1000 non-null   object
 3   posting_type                   1000 non-null   object
 4   number_of_positions            1000 non-null   object
 5   business_title                 1000 non-null   object
 6   civil_service_title            1000 non-null   object
 7   title_classification           1000 non-null   object
 8   title_code_no                  1000 non-null   object
 9   level                          1000 non-null   object
 10  job_category                   1000 non-null   object
 11  full_time_part_time_indicator  985 non-null    object
 12  career_level                   1000 non-null   object
 13  sala

In [42]:
df_cleaned = df_raw.copy()

In [45]:
df_cleaned = df_raw.drop(columns=  ['recruitment_contact', 'work_location_1', 'hours_shift', 'post_until', 'additional_information'])
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   __id                           1000 non-null   object
 1   job_id                         1000 non-null   object
 2   agency                         1000 non-null   object
 3   posting_type                   1000 non-null   object
 4   number_of_positions            1000 non-null   object
 5   business_title                 1000 non-null   object
 6   civil_service_title            1000 non-null   object
 7   title_classification           1000 non-null   object
 8   title_code_no                  1000 non-null   object
 9   level                          1000 non-null   object
 10  job_category                   1000 non-null   object
 11  full_time_part_time_indicator  985 non-null    object
 12  career_level                   1000 non-null   object
 13  sala

In [46]:
CONNECTION_STRING_AZURE_STORAGE = "DefaultEndpointsProtocol=https;AccountName=nycjobs;AccountKey=IuQ3EcwNFwEaSl4kBB7pn6YFkrmux0GxuaqplhfQErilu32k8n7p4lw9ckn65O3VMTvGrUijAgaY+ASt6+1LYA==;EndpointSuffix=core.windows.net"
CONTAINER_AZURE = 'nycjobs'
blob_name = "nycjobs.csv"
# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")


Uploaded nycjobs.csv to Azure Blob Storage in container nycjobs.


In [47]:
# Example usage
if __name__ == "__main__":

    output = StringIO()
    df_raw.to_csv(output, index=False)
    data = output.getvalue()
    output.close()

    # Azure
    azure_connect_str = 'DefaultEndpointsProtocol=https;AccountName=nycjobs;AccountKey=IuQ3EcwNFwEaSl4kBB7pn6YFkrmux0GxuaqplhfQErilu32k8n7p4lw9ckn65O3VMTvGrUijAgaY+ASt6+1LYA==;EndpointSuffix=core.windows.net'
    azure_container_name = 'nycjobs'
    azure_blob_name = 'nycjobs.csv'
    azure_data = 'Hello, Azure!'
    azure_upload_blob(azure_connect_str, azure_container_name, azure_blob_name, azure_data)
    # Download and print the blob content
    azure_blob_content = azure_download_blob(azure_connect_str, azure_container_name, azure_blob_name)
    print(azure_blob_content)


Uploaded to Azure Blob: nycjobs.csv
b'Hello, Azure!'
